##Module 6.5: Dueling DQN / Advantage Network

We implement a dueling - or advantage - (D)DQN based reinforcement learning system with experiential replay to control the cart-pole environment from AI Gym.

We will also rander and record our environment so we can play back videos of how our system is controlling it.

In this module we will pay attention to:
- Examining the value/advantage architecture of the DQN.

We will also set up our class so we can choose to have a DQN or DDQN setup.

Note that we will not spend time tuning hyper-parameters: The purpose is to show how different techniques can be implemented in Keras, not to solve particular data science problems as optimally as possible. Obviously, most techniques include hyper-parameters that need to be tuned for optimal performance.

We will need to install some additional libraries in order to capture and play back video of our reinforcement learning system controlling the environment. We install the libraries now.

In [0]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

And here we set up the display for video playback. Don't worry if you see a warning.

Displaying interactive graphics is a headache in Colab, which is why we choose to capture and playback video. If this does not work for you, you can run the code without the video components

In [0]:
from IPython import display as ipythondisplay
from IPython.display import HTML
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

Now we set up the cart-pole environments using AI Gym.

In [0]:
from gym.wrappers import TimeLimit
from gym.envs.classic_control import CartPoleEnv
env = TimeLimit(CartPoleEnv(),max_episode_steps=251)
env_long = TimeLimit(CartPoleEnv(),max_episode_steps=5001)

Now we set up our reinforcement learning control system. We will use a class for this.

In [0]:
# Import required libraries
import numpy as np

import matplotlib.pyplot as plt
import random
import time
import glob
import io
import base64

from keras.optimizers import Adam
from keras.layers import Input, Dense, Lambda
from keras.layers.merge import Add
from keras.models import Model
import keras.backend as K

from gym.wrappers import Monitor


# Our Dueling (D)DQN Reinforcement Learning Class
class Dueling_DDQN_Brain:
  # Lots of parameters we can initialize
  def __init__ (
          self,
          epsilon_start=.9,
          epsilon_stop=.1,
          epsilon_decay_episodes=500,
          gamma=.9,
          use_DDQN=True, # Here we specify if we use a double network
          update_period=20,
          memory_max=10000,
          batch_size=128,
          state_size=4, 
          action_size=2, 
          hidden_size=256,
          learning_rate=.01
          ):
    self.steps=0
    self.qNet=self.makeDQN(state_size, action_size, hidden_size,learning_rate)
    self.use_DDQN=use_DDQN
    # We make the DDQN anyway - we might change our mind about having one
    self.dqNet=self.makeDQN(state_size, action_size, hidden_size,learning_rate)
    self.update_weights()
    self.epsilon=epsilon_start
    self.epsilon_stop=epsilon_stop
    self.epsilon_decay = self.epsilon_stop / self.epsilon
    self.epsilon_decay = self.epsilon_decay ** (1. / float(epsilon_decay_episodes))
    self.gamma=gamma
    self.memory_max=memory_max
    self.memory=[]
    self.batch_size=batch_size
    self.replay_count=0
    self.update_period=update_period

  # THIS FUNCTION HAS BEEN CHANGED
  # Here we create the DQN we will use
  # This will have a state-value/action-advantage structure
  def makeDQN(self,state_size, action_size, hidden_size,learning_rate):
    # Standard start to the architecture
    inputs = Input(shape=(state_size,))
    h1 = Dense(hidden_size, activation='relu')(inputs)
    h2 = Dense(int(hidden_size/2), activation='relu')(h1)

    # Network splits into value and advantage branches
    # Value branch
    value_dense = Dense(1)(h2)
    value = Lambda(lambda s: K.expand_dims(s[:, 0], -1), output_shape=(action_size,))(value_dense)

    # Advantage branch
    advantage_dense = Dense(action_size)(h2)
    advantage = Lambda(lambda a: a[:, :] - K.mean(a[:, :], keepdims=True),
                              output_shape=(action_size,))(advantage_dense)

    # Sum Value and Advantages to get Q-values
    output = Add()([value, advantage])
    model = Model(input=inputs, output=output)
    model.summary()
    model.compile(loss='mse', optimizer=Adam(lr=learning_rate))
    return model

  # This function decays epsilon according to our parameters
  def update_epsilon(self):
    if self.epsilon > self.epsilon_stop:
      self.epsilon *= self.epsilon_decay

  # Here we copy the weight values of qNet into dqNet
  def update_weights(self):
    self.dqNet.set_weights(self.qNet.get_weights())

  # We make a decision based on which of our estimated
  # Q-values is highest for a given state
  def decide (self, state):
    Qs = self.qNet.predict(state)
    return np.argmax(Qs)            
  
  # We store a SARS datum in memory.
  def add_to_memory (self,state,action,reward,next_state,done):
    if len(self.memory)==self.memory_max:
      self.memory.pop(0)
    self.memory.append((state,action,reward,next_state,done))

  # THIS FUNCTION HAS BEEN CHANGED
  # This is where we get the q-value for
  # a state for training purposes.
  def get_target_q_value (self,next_state,reward,done):
    if done:
      return reward
    # CHANGE:
    # We use DDQN or DQN approach depending on setup      
    elif self.use_DDQN:
      # DDQN
      # We use qNet to determine which action to take in
      # the next state.
      action = np.argmax(self.qNet.predict(next_state)[0])
      # But then we use dqNet not qNet to predict q_value 
      # of this action being taken in the next state
      q_value = self.dqNet.predict(next_state)[0,action]
      return reward + self.gamma * q_value    
    else:
      # DQN
      q_value = np.amax(self.qNet.predict(next_state)[0])
      return reward + self.gamma * q_value      
  
  # THIS FUNCTION HAS BEEN CHANGED
  # This is where we fetch data from memory and prepare it 
  # as a training batch
  def fetch_batch_from_memory (self):
    # Take a random sample of data store in memory
    memory_sample = random.sample(self.memory, min(self.batch_size,len(self.memory)))
    
    # Construct training batch from memory store
    states = np.zeros((self.batch_size,4))
    q_values = np.zeros((self.batch_size,2))
    i=0
    for state, action, reward, next_state, done in memory_sample:
      # CHANGE:
      # We use DDQN or DQN approach depending on setup      
      if self.use_DDQN:
        # DDQN
        # Get q-values for state using dqNet, not qNet
        q_values[i] = self.dqNet.predict(state)[0]      
      else:
        # DQN
        # Get q-values for state using qNet
        q_values[i] = self.qNet.predict(state)[0]      

      # Get current estimate of action's q-value
      q_values[i,action] = self.get_target_q_value(next_state, reward, done)

      # Add state and q-values to batch
      states[i]=state[0]
      i+=1

    return states,q_values

  # This is where we train the network, by 'replaying'
  # data stored in memory
  def replay (self):
    # We only start training once their is at least enough
    # data in memory to create a full batch
    if self.batch_size <= len(self.memory):
      # We prepare the input and target batches
      inputs,targets = self.fetch_batch_from_memory()
      # We adjust the qNet network parameters
      self.qNet.fit(inputs,targets,batch_size=self.batch_size,epochs=1,verbose=0)
      # We decay epsilon leading us to exploit more and explore less.
      self.update_epsilon()
      # If using DDQN, then periodically update dqNet
      self.replay_count+=1
      if self.use_DDQN and self.replay_count > 0 and self.replay_count % self.update_period == 0:
        self.update_weights()

  # Here we determine what action we should take in a given state.
  # Notice that we only explore when training.
  def act (self,env,state,train):
    # Determine if we should explore
    if train and np.random.rand() <= self.epsilon:
      # Make a random action
      return env.action_space.sample()
    # Get action from Q-estimates
    return self.decide(state)
  
  # This runs a complete episode
  def control_episode (self,env,train,render=False):
    # We will track total reward over the episode
    total_reward=0

    # Start new episode
    env.reset()

    # Initial random move to get pole and cart moving
    state, reward, done, _ = env.step(env.action_space.sample())
    state = np.reshape(state, [1, 4])
    
    # The control loop for taking steps
    done = False
    while not done:
      # If we want, we render the environment (create an image)
      if render:
        env.render()

      # Determine action
      action = self.act(env,state,train)

      # Take action, get new state and reward
      next_state, reward, done, _ = env.step(action)
      next_state = np.reshape(next_state, [1, 4])
      
      # CHANGE
      # Add information to memory AND train using replay
      # Note we have moved training to every turn!
      if train:
        self.add_to_memory(state,action,reward,next_state,done)
        self.replay()

      # Update local variables
      state=next_state
      total_reward += reward

    # the episode has ended, so record result
    return total_reward

  # Here we control multiple episodes
  def control (self,env,num_episodes,max_steps,train,report_every=100):
    # We track the total rewards in each episode
    final_rewards=np.zeros(num_episodes)
    # We will also time how long things are taking.
    time_start=time.clock()
    
    # We run the desired number of episodes
    for ep in range(num_episodes):
      final_rewards[ep]=self.control_episode(env,train,False)

      # And periodically we print out a report of how we are performing
      if ep!=0 and ep % report_every == 0:
        time_end=time.clock()
        mean_reward=np.sum(final_rewards[ep-report_every:ep+1])/report_every
        print('Episode: {}'.format(ep),
          'Recent Average Rewards: {}'.format(mean_reward),
          'Epsilon: {:.4f}'.format(self.epsilon),
          'Time: {:.2f}'.format(time_end-time_start))
        time_start=time_end
        # We classify the problem as solved if we get an average
        # reward of over 190 during a reporting period.
        # This is actually a little low - the system will learn
        # to balance the pole, but not to keep the cart centered.
        if mean_reward>190:
          print("Problem is considered solved!!!")
          return final_rewards

    print("Problem was not solved.") 
    return final_rewards

  # This will record and show a video of an episode
  # (if the video display code is run)
  def display_episode(self,env):
    wrapped_env=wrap_env(env)
    self.control_episode(wrapped_env,False,True)
    wrapped_env.close()
    show_video()

# This is a function used in showing videos
def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

# This is a function used when recording videos
def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env


Using TensorFlow backend.


In [0]:
dueling_dqn_brain=Dueling_DDQN_Brain(use_DDQN=False,hidden_size=24,learning_rate=.001,gamma=.99,batch_size=12,epsilon_decay_episodes=1000,epsilon_stop=.01)




Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 4)            0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 24)           120         input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 12)           300         dense_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 1)            13          dense_2[0][0]                    
_________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:75: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ad...)`


In [0]:
dueling_dqn_brain.display_episode(env_long)


/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Trying to monitor an environment which has no 'spec' set. This usually means you did not create it via 'gym.make', and is recommended only for advanced users.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [0]:
dqn_R=dueling_dqn_brain.control(env,5000,200,True,50)



Episode: 50 Recent Average Rewards: 12.32 Epsilon: 0.0591 Time: 12.60
Episode: 100 Recent Average Rewards: 10.5 Epsilon: 0.0100 Time: 10.33
Episode: 150 Recent Average Rewards: 11.6 Epsilon: 0.0100 Time: 11.30
Episode: 200 Recent Average Rewards: 11.86 Epsilon: 0.0100 Time: 11.46
Episode: 250 Recent Average Rewards: 10.32 Epsilon: 0.0100 Time: 9.99
Episode: 300 Recent Average Rewards: 9.82 Epsilon: 0.0100 Time: 9.62
Episode: 350 Recent Average Rewards: 9.94 Epsilon: 0.0100 Time: 9.51
Episode: 400 Recent Average Rewards: 9.78 Epsilon: 0.0100 Time: 9.62
Episode: 450 Recent Average Rewards: 9.5 Epsilon: 0.0100 Time: 10.17
Episode: 500 Recent Average Rewards: 9.72 Epsilon: 0.0100 Time: 9.32
Episode: 550 Recent Average Rewards: 10.06 Epsilon: 0.0100 Time: 9.71
Episode: 600 Recent Average Rewards: 10.32 Epsilon: 0.0100 Time: 9.75
Episode: 650 Recent Average Rewards: 12.22 Epsilon: 0.0100 Time: 11.85
Episode: 700 Recent Average Rewards: 12.14 Epsilon: 0.0100 Time: 11.69
Episode: 750 Recent 

In [0]:
dueling_dqn_brain.display_episode(env_long)

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Trying to monitor an environment which has no 'spec' set. This usually means you did not create it via 'gym.make', and is recommended only for advanced users.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Trying to monitor an environment which has no 'spec' set. This usually means you did not create it via 'gym.make', and is recommended only for advanced users.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [0]:
dueling_ddqn_brain=Dueling_DDQN_Brain(hidden_size=24,learning_rate=.001,gamma=.99,batch_size=12,epsilon_decay_episodes=1000,epsilon_stop=.01)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:75: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ad...)`


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 4)            0                                            
__________________________________________________________________________________________________
dense_9 (Dense)                 (None, 24)           120         input_3[0][0]                    
__________________________________________________________________________________________________
dense_10 (Dense)                (None, 12)           300         dense_9[0][0]                    
__________________________________________________________________________________________________
dense_11 (Dense)                (None, 1)            13          dense_10[0][0]                   
____________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:75: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ad...)`


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 4)            0                                            
__________________________________________________________________________________________________
dense_9 (Dense)                 (None, 24)           120         input_3[0][0]                    
__________________________________________________________________________________________________
dense_10 (Dense)                (None, 12)           300         dense_9[0][0]                    
__________________________________________________________________________________________________
dense_11 (Dense)                (None, 1)            13          dense_10[0][0]                   
____________________________________________________________________________________________

In [0]:
ddqn_R=dueling_ddqn_brain.control(env,5000,200,True,20)

Episode: 20 Recent Average Rewards: 15.75 Epsilon: 0.2292 Time: 9.87
Episode: 40 Recent Average Rewards: 10.65 Epsilon: 0.0932 Time: 5.97
Episode: 60 Recent Average Rewards: 10.3 Epsilon: 0.0384 Time: 5.75
Episode: 80 Recent Average Rewards: 10.8 Epsilon: 0.0151 Time: 6.15
Episode: 100 Recent Average Rewards: 11.9 Epsilon: 0.0100 Time: 6.75
Episode: 120 Recent Average Rewards: 11.1 Epsilon: 0.0100 Time: 6.28
Episode: 140 Recent Average Rewards: 11.15 Epsilon: 0.0100 Time: 6.30
Episode: 160 Recent Average Rewards: 10.55 Epsilon: 0.0100 Time: 5.88
Episode: 180 Recent Average Rewards: 10.5 Epsilon: 0.0100 Time: 5.80
Episode: 200 Recent Average Rewards: 10.4 Epsilon: 0.0100 Time: 5.82
Episode: 220 Recent Average Rewards: 10.3 Epsilon: 0.0100 Time: 5.81
Episode: 240 Recent Average Rewards: 10.3 Epsilon: 0.0100 Time: 5.69
Episode: 260 Recent Average Rewards: 10.2 Epsilon: 0.0100 Time: 5.61
Episode: 280 Recent Average Rewards: 10.5 Epsilon: 0.0100 Time: 5.69
Episode: 300 Recent Average Reward

In [0]:
dueling_ddqn_brain.display_episode(env_long)

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Trying to monitor an environment which has no 'spec' set. This usually means you did not create it via 'gym.make', and is recommended only for advanced users.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Trying to monitor an environment which has no 'spec' set. This usually means you did not create it via 'gym.make', and is recommended only for advanced users.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [0]:
display.stop()

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] oserror=None return_code=0 stdout="" stderr="" timeout_happened=False>

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] oserror=None return_code=0 stdout="" stderr="" timeout_happened=False>